### Лабораторна работа 6.
Выполнили студенты группы **6404**
- Бахаев Дмитрий В.
- Сагалов Никита Д.

**6.1 Написать функции кодирования и декодирования для циклического кода (7,4) с порождающим многочленом 𝑔(𝑥) = 1 + 𝑥2 + 𝑥3, исправляющего однократные ошибки и провести исследование этого кода для одно-, двух- и трёхкратных ошибок.**

In [1]:
import numpy as np

In [2]:
def create_G(n, m, g_x):
    G = np.zeros((m, n), dtype=int)
    for i in range(m):
        G[i, i: i + g_x.size] = g_x
    for i in range(1, m):
        for j in range(i):
            if G[j][i] == 1:
                G[j] ^= G[i]
    return G


def vec_to_num(v):
    return (np.array([2 ** i for i in range(v.size)]) * v).sum()


def find_remainder(bin_pol_num_1, bin_pol_num_2, n, m):
    bin_pol_num_2 <<= m
    for i in range(1, m + 1):
        bin_pol_num_2 >>= 1
        if bin_pol_num_1 >= 1 << (n - i):
            bin_pol_num_1 ^= bin_pol_num_2
    return bin_pol_num_1


def decode_word(w, m, t, g_x_num):
    n = w.size
    w_num = vec_to_num(w)
    w_num = find_remainder(w_num, g_x_num, n, m)
    if w_num == 0:
        return w.copy()[:m]
    k = 0

    while k > -n:
        if w_num < 1 << t and w_num % 2 == 1:  #
            break
        w_num <<= 1
        if w_num >= 1 << (n - m):
            w_num ^= g_x_num
        k -= 1

    w_fix = w.copy()
    while w_num != 0:
        if w_num % 2 == 1:
            w_fix[k] ^= 1
        k += 1
        w_num >>= 1
    return w_fix[:m]


In [3]:
n = 7
m = 4
t = 1
g_x = np.array([1, 0, 1, 1])
g_x_num = vec_to_num(g_x)
g_x_num

13

In [4]:
G = create_G(n, m, g_x)
G

array([[1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1]])

In [5]:
word = np.array([1, 0, 0, 1])
v = np.dot(word, G) % 2
v

array([1, 0, 0, 1, 1, 1, 0], dtype=int32)

Создаём векторы ошибок:  
- Двухразрядную ошибку (`err2`), трёхразрядную ошибку (`err3`) и массив всех возможных одноразрядных ошибок.  
- Генерируем ошибочные кодовые слова, добавляя ошибки к исходному кодовому слову.

In [6]:
err2 = np.zeros_like(v, dtype=int)
err2[1] = err2[2] = 1
err3 = err2.copy()
err3[3] = 1

w_err2 = v ^ err2
w_err3 = v ^ err3
w_err1_array = [v.copy() ^ err1 for err1 in np.eye(n, dtype=int)]
w_err1_array

[array([0, 0, 0, 1, 1, 1, 0], dtype=int32),
 array([1, 1, 0, 1, 1, 1, 0], dtype=int32),
 array([1, 0, 1, 1, 1, 1, 0], dtype=int32),
 array([1, 0, 0, 0, 1, 1, 0], dtype=int32),
 array([1, 0, 0, 1, 0, 1, 0], dtype=int32),
 array([1, 0, 0, 1, 1, 0, 0], dtype=int32),
 array([1, 0, 0, 1, 1, 1, 1], dtype=int32)]

Декодируем исходное кодовое слово и проверяем, совпадает ли оно с исходным информационным словом. Затем декодируем кодовые слова с одноразрядными ошибками, проверяя корректность их исправления.

In [7]:
dec_word = decode_word(v, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [8]:
for w in w_err1_array:
    dec_word = decode_word(w, m, t, g_x_num)
    print(np.array_equal(dec_word, word))

True
True
True
True
True
True
True


Декодируем кодовые слова с двухразрядной и трёхразрядной ошибками и проверяем, удалось ли восстановить исходное информационное слово.

In [9]:
dec_word = decode_word(w_err2, m, t, g_x_num)
np.array_equal(dec_word, word)

False

In [10]:
dec_word = decode_word(w_err3, m, t, g_x_num)
np.array_equal(dec_word, word)

False

**6.2 Написать функции кодирования и декодирования для циклического кода (15,9) с порождающим многочленом 𝑔(𝑥) = 1 + 𝑥3 + 𝑥4 + 𝑥5 + 𝑥6, исправляющего пакеты ошибок кратности 3 и провести исследование этого кода для пакетов ошибок длины 1, 2, 3 и 4. Обратите внимание, что пакет ошибок длины t не означает, что все разряды в пределах этого пакета изменятся (см. лекции).**

In [11]:
n = 15
m = 9
t = 3
g_x = np.array([1, 1, 1, 1, 0, 0, 1])
g_x_num = vec_to_num(g_x)
g_x_num

79

In [12]:
G = create_G(n, m, g_x)
G

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1]])

In [13]:
word = np.array([1, 1, 1, 0, 0, 0, 0, 0, 0])
v = np.dot(word, G) % 2
v

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32)

Создаём векторы ошибок для кодового слова:  
- Двухразрядную ошибку (`err2`), трёхразрядную ошибку (`err3`) и четырёхразрядную ошибку (`err4`).  
- Генерируем ошибочные кодовые слова, добавляя эти ошибки к исходному кодовому слову, а также массив кодовых слов с одноразрядными ошибками.

In [14]:
err2 = np.zeros_like(v, dtype=int)
err2[1] = err2[2] = 1
err3 = err2.copy()
err3[3] = 1
err4 = np.zeros_like(v, dtype=int)
err4[6] = err4[7] = err4[9] = 1

w_err2 = v ^ err2
w_err3 = v ^ err3
w_err4 = v ^ err4
w_err1_array = [v.copy() ^ err1 for err1 in np.eye(n, dtype=int)]
w_err1_array

[array([0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=int32),
 array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0

Декодируем исходное кодовое слово и проверяем, совпадает ли оно с исходным информационным словом. Затем декодируем кодовые слова с одноразрядными ошибками, проверяя корректность их исправления.

In [15]:
dec_word = decode_word(v, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [16]:
for w in w_err1_array:
    dec_word = decode_word(w, m, t, g_x_num)
    print(np.array_equal(dec_word, word))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


Декодируем кодовые слова с двух-, трёх- и четырёхразрядной ошибками и проверяем, удалось ли восстановить исходное информационное слово.

In [17]:
dec_word = decode_word(w_err2, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [18]:
dec_word = decode_word(w_err3, m, t, g_x_num)
np.array_equal(dec_word, word)

True

In [19]:
dec_word = decode_word(w_err4, m, t, g_x_num)
np.array_equal(dec_word, word)

False